In [1]:
! pip install rouge_score evaluate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a1ec11ca6d2c8b2f7534fddba698f61c7fa7dadac51f3f5352ed52313b2c23e3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: 

In [2]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os
import torch

from datasets import Dataset, load_metric

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load the data

In [3]:
# Paths to the files
LED_large = '/content/drive/MyDrive/W266_Project/Data/Results/LED_LARGE_2/led_large_lora_predictions.csv'
LED_Bio = '/content/drive/MyDrive/W266_Project/Data/Results/led_large_lora_bio_all_ner_predictions.csv'
LED_Clinical = '/content/drive/MyDrive/W266_Project/Data/Results/led_large_clinical_all_ner_predictions.csv'
LED_keybert = '/content/drive/MyDrive/W266_Project/Data/Results/led_large_keyword_predictions.csv'

# Load the datasets
LED_large = pd.read_csv(LED_large)
LED_Bio = pd.read_csv(LED_Bio)
LED_Clinical = pd.read_csv(LED_Clinical)
LED_keybert = pd.read_csv(LED_keybert)


# Check the shape
print("Shape of LED_large:")
print(LED_large.shape)
print("Shape of LED_Bio:")
print(LED_Bio.shape)
print("Shape of LED_Clinical:")
print(LED_Clinical.shape)
print("Shape of LED_keybert:")
print(LED_keybert.shape)


Shape of LED_large:
(180, 3)
Shape of LED_Bio:
(180, 3)
Shape of LED_Clinical:
(180, 3)
Shape of LED_keybert:
(180, 3)


In [4]:
LED_large.head(2)

,Dialogue,Reference,Prediction
0,"doctor: Good morning, Mr. A. Thank you for com...",SUBJECTIVE CHIEF COMPLAINT New patient evaluat...,SUBJECTIVE CHIEF COMPLAINT Malignant meningiom...
1,"doctor: Hello, I'm Dr. doctor's name. I unders...","SUBJECTIVE CHIEF COMPLAINT Fever, fussiness, a...",SUBJECTIVE CHIEF COMPLAINT The patient is a pl...


In [5]:
LED_Bio.head(2)

,Dialogue,Reference,Prediction
0,"doctor: Good morning, Mr. A. Thank you for com...",SUBJECTIVE CHIEF COMPLAINT New patient evaluat...,SUBJECTIVE CHIEF COMPLAINT Malignant meningiom...
1,"doctor: Hello, I'm Dr. doctor's name. I unders...","SUBJECTIVE CHIEF COMPLAINT Fever, fussiness, a...",SUBJECTIVE CHIEF COMPLAINT Fussy and crying in...


In [6]:
LED_Clinical.head(2)

,Dialogue,Reference,Prediction
0,"doctor: Good morning, Mr. A. Thank you for com...",SUBJECTIVE CHIEF COMPLAINT New patient evaluat...,SUBJECTIVE CHIEF COMPLAINT Malignant meningiom...
1,"doctor: Hello, I'm Dr. doctor's name. I unders...","SUBJECTIVE CHIEF COMPLAINT Fever, fussiness, a...",SUBJECTIVE CHIEF COMPLAINT Infant with cutis m...


In [7]:
LED_keybert.head(2)

,Dialogue,Reference,Prediction
0,"doctor: Good morning, Mr. A. Thank you for com...",SUBJECTIVE CHIEF COMPLAINT New patient evaluat...,SUBJECTIVE CHIEF COMPLAINT Malignant meningiom...
1,"doctor: Hello, I'm Dr. doctor's name. I unders...","SUBJECTIVE CHIEF COMPLAINT Fever, fussiness, a...",SUBJECTIVE CHIEF COMPLAINT Baby has been irrit...


# Get dialogue  and rederence list

In [8]:
references = LED_large["Reference"].tolist()

In [9]:
len(references)

180

In [10]:
LED_baseline_predictions = LED_large["Prediction"].tolist()

In [11]:
len(LED_baseline_predictions)

180

In [12]:
LED_Bio_predictions = LED_Bio["Prediction"].tolist()

In [13]:
LED_Clinical_predictions = LED_Clinical["Prediction"].tolist()

In [14]:
LED_keybert_predictions = LED_keybert["Prediction"].tolist()

In [15]:
references[0]

"SUBJECTIVE CHIEF COMPLAINT New patient evaluation. HISTORY OF PRESENT ILLNESS The patient is a pleasant age-year-old male who presents for a new patient evaluation. He reports a history of malignant meningioma with progression of disease and significant brain edema. He has undergone two surgeries in the past, the first in July 2006 and the second in August 2007. After that, he started having seizures and hemiparesis. He has been in and out of the hospital and developed C. diff. He is currently in a nursing home because he is unable to care for himself. The patient's current medications include Keppra, Decadron, oxycodone, aspirin, Prilosec, Dilantin, and Flagyl. He denies any allergies. ASSESSMENT AND PLAN: 1. Malignant meningioma with progression of disease and significant brain edema Assessment: - Patient has worsening symptoms of seizures, hemiparesis, and confusion due to progression of malignant meningioma and brain edema. - Patient has exhausted all curative treatment options. P

In [16]:
LED_baseline_predictions[0]

"SUBJECTIVE CHIEF COMPLAINT Malignant meningioma. HISTORY OF PRESENT ILLNESS The patient is a pleasant age-year-old male who presents to the clinic today for a new patient evaluation. The patient reports that he is unable to care for himself due to his medical history, including two surgeries in the past. The first was in July of 2006, and the second was in August of 2007. After that, he developed seizures and hemiparesis. He has been in and out of the hospital, and has developed C. diff. A. He is currently taking Keppra, Decadron, oxycodone, aspirin, Prilosec, Dilantin, and Flagyl for his symptoms. ASSESSMENT AND PLAN: 1. Malignant malignant meneningioma Assessment: The patient presents with progression of disease and significant brain edema. The likelihood of benefit from surgery is small, and there's a good chance that the patient wouldn't be able to recover. The best we can do is try to manage symptoms and make you as comfortable as possible. Plan: - Continue to be on the current m

In [17]:
LED_Bio_predictions[0]

"SUBJECTIVE CHIEF COMPLAINT Malignant meningioma with progression of disease and significant brain edema. HISTORY OF PRESENT ILLNESS The patient is a pleasant age-year-old male who presents for a new patient evaluation. He presents for evaluation because he has malignant meneningioma. He has been in and out of the hospital, and has had two surgeries in the past. The first was in July 2006, and the second was in August 2007. After that, he developed seizures and hemiparesis. He is currently in a nursing home because he is unable to care for care for himself. The patient has a history of Keppra, Decadron, oxycodone, aspirin, Prilosec, Dilantin, and Flagyl. He also has C. diff, which is causing his symptoms. ASSESSMENT AND PLAN: 1. Malignant malignant malignioma Assessment: The patient presents for new patient assessment. He presented for a second surgery in August of 2007. The likelihood of benefit from surgery is small, and there's a good chance that he wouldn't be able to recover from 

In [18]:
LED_Clinical_predictions[0]

"SUBJECTIVE CHIEF COMPLAINT Malignant meningioma. HISTORY OF PRESENT ILLNESS The patient is a pleasant age-year-old male who presents for evaluation. He presents to the clinic today for a new patient evaluation. The patient states that he is a patient who has been in and out of the hospital for several years due to his medical history. He is currently in a nursing home because he is unable to care for care for himself. He has seizures and hemiparesis, and hemi par esis. His medical history includes two surgeries in the past. The first was in July of 2006, and the second was in August of 2007. After that, he developed C. diff, which caused seizures. He also has significant brain edema, which is causing his symptoms, such as the seizures, hemipareis and confusion. ASSESSMENT AND PLAN: 1. Malignant malignant menignioma Assessment: The patient has a malignant malign menignum with progression, disease, and significant swelling in his brain. The tumor has grown and is causing swelling in the

In [19]:
LED_keybert_predictions[0]

"SUBJECTIVE CHIEF COMPLAINT Malignant meningioma with progression of disease and significant brain edema. HISTORY OF PRESENT ILLNESS The patient is a pleasant age-year-old male who presents for evaluation. He presents to the clinic for a new patient evaluation. His medical history includes seizures and hemiparesis. He has been in and out of the hospital, and he has had two surgeries in the past. He is currently in a nursing home because he's unable to care for care for himself. He was diagnosed with malignant meneningioma, which is causing swelling in his brain. He also has a history of C. diff. The patient has been taking Keppra, Decadron, oxycodone, aspirin, Prilosec, Dilantin, and Flagyl for his symptoms. He developed C diff in August of 2007, which caused him to develop seizures. He's currently in the nursing home due to his condition. ASSESSMENT AND PLAN: 1. Malignant malignant malignioma Assessment: The patient's medical history is a complete medical history. His symptoms include

# Rough Score

In [20]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(LED_baseline_predictions, references):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")



Average ROUGE-1 Recall: 0.5006
Average ROUGE-1 Precision: 0.6519
Average ROUGE-1 F1: 0.5514
Average ROUGE-2 Recall: 0.2833
Average ROUGE-2 Precision: 0.3668
Average ROUGE-2 F1: 0.3112
Average ROUGE-L Recall: 0.2804
Average ROUGE-L Precision: 0.3680
Average ROUGE-L F1: 0.3099


In [21]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(LED_Clinical_predictions, references):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_cli = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_cli = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_cli = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_cli = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_cli = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_cli = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_cli = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_cli = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_cli = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_cli:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_cli:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_cli:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_cli:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_cli:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_cli:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_cli:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_cli:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_cli:.4f}")



Average ROUGE-1 Recall: 0.5067
Average ROUGE-1 Precision: 0.6799
Average ROUGE-1 F1: 0.5631
Average ROUGE-2 Recall: 0.2980
Average ROUGE-2 Precision: 0.3990
Average ROUGE-2 F1: 0.3306
Average ROUGE-L Recall: 0.3003
Average ROUGE-L Precision: 0.4084
Average ROUGE-L F1: 0.3355


In [22]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(LED_Bio_predictions, references):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_bio = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_bio = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_bio = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_bio = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_bio = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_bio = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_bio = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_bio = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_bio = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_bio:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_bio:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_bio:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_bio:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_bio:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_bio:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_bio:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_bio:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_bio:.4f}")



Average ROUGE-1 Recall: 0.4874
Average ROUGE-1 Precision: 0.6825
Average ROUGE-1 F1: 0.5542
Average ROUGE-2 Recall: 0.2794
Average ROUGE-2 Precision: 0.3916
Average ROUGE-2 F1: 0.3177
Average ROUGE-L Recall: 0.2777
Average ROUGE-L Precision: 0.3939
Average ROUGE-L F1: 0.3173


In [23]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(LED_keybert_predictions, references):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_key = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_key = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_key = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_key = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_key = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_key = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_key = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_key = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_key = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_key:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_key:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_key:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_key:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_key:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_key:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_key:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_key:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_key:.4f}")

Average ROUGE-1 Recall: 0.4782
Average ROUGE-1 Precision: 0.6712
Average ROUGE-1 F1: 0.5439
Average ROUGE-2 Recall: 0.2690
Average ROUGE-2 Precision: 0.3776
Average ROUGE-2 F1: 0.3060
Average ROUGE-L Recall: 0.2671
Average ROUGE-L Precision: 0.3806
Average ROUGE-L F1: 0.3057


# get a dataframe to show all metric

In [25]:
import pandas as pd


# Define the scores for each model
scores = {
    'led_baseline_long': {
        'rouge1_recall': round(avg_rouge1_recall_baseline, 4),
        'rouge1_precision': round(avg_rouge1_precision_baseline, 4),
        'rouge1_f1': round(avg_rouge1_f1_baseline, 4),
        'rouge2_recall': round(avg_rouge2_recall_baseline, 4),
        'rouge2_precision': round(avg_rouge2_precision_baseline, 4),
        'rouge2_f1': round(avg_rouge2_f1_baseline, 4),
        'rougeL_recall': round(avg_rougeL_recall_baseline, 4),
        'rougeL_precision': round(avg_rougeL_precision_baseline, 4),
        'rougeL_f1': round(avg_rougeL_f1_baseline, 4),
    },
    'led_bio_long': {
        'rouge1_recall': round(avg_rouge1_recall_bio, 4),
        'rouge1_precision': round(avg_rouge1_precision_bio, 4),
        'rouge1_f1': round(avg_rouge1_f1_bio, 4),
        'rouge2_recall': round(avg_rouge2_recall_bio, 4),
        'rouge2_precision': round(avg_rouge2_precision_bio, 4),
        'rouge2_f1': round(avg_rouge2_f1_bio, 4),
        'rougeL_recall': round(avg_rougeL_recall_bio, 4),
        'rougeL_precision': round(avg_rougeL_precision_bio, 4),
        'rougeL_f1': round(avg_rougeL_f1_bio, 4),
    },
    'led_clinical_long': {
        'rouge1_recall': round(avg_rouge1_recall_cli, 4),
        'rouge1_precision': round(avg_rouge1_precision_cli, 4),
        'rouge1_f1': round(avg_rouge1_f1_cli, 4),
        'rouge2_recall': round(avg_rouge2_recall_cli, 4),
        'rouge2_precision': round(avg_rouge2_precision_cli, 4),
        'rouge2_f1': round(avg_rouge2_f1_cli, 4),
        'rougeL_recall': round(avg_rougeL_recall_cli, 4),
        'rougeL_precision': round(avg_rougeL_precision_cli, 4),
        'rougeL_f1': round(avg_rougeL_f1_cli, 4),
    },
    'led_key_long': {
        'rouge1_recall': round(avg_rouge1_recall_key, 4),
        'rouge1_precision': round(avg_rouge1_precision_key, 4),
        'rouge1_f1': round(avg_rouge1_f1_key, 4),
        'rouge2_recall': round(avg_rouge2_recall_key, 4),
        'rouge2_precision': round(avg_rouge2_precision_key, 4),
        'rouge2_f1': round(avg_rouge2_f1_key, 4),
        'rougeL_recall': round(avg_rougeL_recall_key, 4),
        'rougeL_precision': round(avg_rougeL_precision_key, 4),
        'rougeL_f1': round(avg_rougeL_f1_key, 4),
    }
}


# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(scores, orient='index')

# Reset the index to have a column for the model names
df.reset_index(inplace=True)
df.rename(columns={'index': 'model'}, inplace=True)

df


,model,rouge1_recall,rouge1_precision,rouge1_f1,rouge2_recall,rouge2_precision,rouge2_f1,rougeL_recall,rougeL_precision,rougeL_f1
0,led_baseline_long,0.5006,0.6519,0.5514,0.2833,0.3668,0.3112,0.2804,0.3680,0.3099
1,led_bio_long,0.4874,0.6825,0.5542,0.2794,0.3916,0.3177,0.2777,0.3939,0.3173
2,led_clinical_long,0.5067,0.6799,0.5631,0.2980,0.3990,0.3306,0.3003,0.4084,0.3355
3,led_key_long,0.4782,0.6712,0.5439,0.2690,0.3776,0.3060,0.2671,0.3806,0.3057


Analysis


In [26]:
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

biggest_difference_example = None
biggest_difference_score = 0

# Calculate ROUGE-1 scores and find the biggest difference
for i, (ref, base_pred, clinical_pred) in enumerate(zip(references, LED_baseline_predictions, LED_Clinical_predictions)):
    score_baseline = scorer.score(ref, base_pred)['rouge1'].fmeasure
    score_clinical = scorer.score(ref, clinical_pred)['rouge1'].fmeasure

    # Calculate the absolute difference
    difference = abs(score_baseline - score_clinical)

    # Compare differences
    if difference > biggest_difference_score:
        biggest_difference_score = difference
        biggest_difference_example = (i, score_baseline, score_clinical, difference)

# Output the example with the biggest ROUGE-1 score difference
example_index, baseline_score, clinical_score, difference = biggest_difference_example
print(f"The biggest ROUGE-1 F1 score difference is {difference:.4f} for example {example_index}.")
print(f"Baseline score: {baseline_score:.4f}, Clinical score: {clinical_score:.4f}")


The biggest ROUGE-1 F1 score difference is 0.1949 for example 129.
Baseline score: 0.3698, Clinical score: 0.5648


In [28]:
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True)

biggest_difference_example = None
biggest_difference_score = 0

# Calculate ROUGE-2 scores and find the biggest difference
for i, (ref, base_pred, clinical_pred) in enumerate(zip(references, LED_baseline_predictions, LED_Clinical_predictions)):
    score_baseline = scorer.score(ref, base_pred)['rouge2'].fmeasure
    score_clinical = scorer.score(ref, clinical_pred)['rouge2'].fmeasure

    # Calculate the absolute difference
    difference = abs(score_baseline - score_clinical)

    # Compare differences
    if difference > biggest_difference_score:
        biggest_difference_score = difference
        biggest_difference_example = (i, score_baseline, score_clinical, difference)

# Output the example with the biggest ROUGE-2 score difference
example_index, baseline_score, clinical_score, difference = biggest_difference_example
print(f"The biggest ROUGE-2 F1 score difference is {difference:.4f} for example {example_index}.")
print(f"Baseline score: {baseline_score:.4f}, Clinical score: {clinical_score:.4f}")

The biggest ROUGE-2 F1 score difference is 0.1984 for example 146.
Baseline score: 0.2335, Clinical score: 0.4320


In [30]:
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

biggest_difference_example = None
biggest_difference_score = 0

# Calculate ROUGE-L scores and find the biggest difference
for i, (ref, base_pred, clinical_pred) in enumerate(zip(references, LED_baseline_predictions, LED_Clinical_predictions)):
    score_baseline = scorer.score(ref, base_pred)['rougeL'].fmeasure
    score_clinical = scorer.score(ref, clinical_pred)['rougeL'].fmeasure

    # Calculate the absolute difference
    difference = abs(score_baseline - score_clinical)

    # Compare differences
    if difference > biggest_difference_score:
        biggest_difference_score = difference
        biggest_difference_example = (i, score_baseline, score_clinical, difference)

# Output the example with the biggest ROUGE-L score difference
example_index, baseline_score, clinical_score, difference = biggest_difference_example
print(f"The biggest ROUGE-L F1 score difference is {difference:.4f} for example {example_index}.")
print(f"Baseline score: {baseline_score:.4f}, Clinical score: {clinical_score:.4f}")

The biggest ROUGE-L F1 score difference is 0.2216 for example 37.
Baseline score: 0.2037, Clinical score: 0.4253
